In [2]:
# https://towardsdatascience.com/multi-class-text-classification-with-doc2vec-logistic-regression-9da9947b43f4

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
from pathlib import Path
import pandas as pd
import numpy as np

In [5]:
import os
from os import listdir

In [6]:
DATA = "/kaggle/input"

In [7]:
# explore the input data
# for dirname, _, filenames in os.walk(DATA):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

In [8]:
train_directory = os.path.join(DATA,'feedback-prize-2021','train')

In [9]:
test_directory = os.path.join(DATA,'feedback-prize-2021','test')

In [10]:
glove_directory = os.path.join(DATA,'glovedata')

In [11]:
root_directory = os.path.join(DATA,'feedback-prize-2021')

In [12]:
# explore the word2vec datasets
for dirname, _, filenames in os.walk(glove_directory):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/glovedata/glove.6B.200d.txt
/kaggle/input/glovedata/glove.6B.50d.txt
/kaggle/input/glovedata/glove.6B.100d.txt


In [13]:
train_filename = "train.csv"

In [14]:
train_filepath = os.path.join(root_directory,train_filename)

In [15]:
essay_files = listdir(train_directory)

In [16]:
len(essay_files)

15594

In [17]:
# to bring in the training set spreadsheet (.csv)

In [18]:
train_df = pd.read_csv(train_filepath)

In [19]:
train_df.head()

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...


In [20]:
print(len(train_df))

144293


In [21]:
def retrieveWordage(essay_id):
  essay_filename = essay_id + ".txt"
  essay_filepath = os.path.join(train_directory,essay_filename)
  with open(essay_filepath) as file:
    file_contents = file.read()
  #try:
  #  essay_text = pd.read_table(essay_filepath,header=0,engine='python',error_bad_lines=False)
  #except:
  #  return None
  #essay_text.columns = ['narrative']
  #return essay_text
  essay_text = []
  essay_line = ""
  for c in file_contents:
    if c == '\n':
      if len(essay_line) > 1:
        essay_text.append(essay_line)
      essay_line = ""
    else:
      essay_line += c
  essay_text.append(essay_line)
  essay_pd = pd.DataFrame(essay_text)
  essay_pd.columns = ['narrative']
  return essay_pd

In [22]:
wordage_test = retrieveWordage("0027FC00C35B")
print(wordage_test)

                                            narrative
0                                    Phones & Driving
1   Waking up from a wonderful dream, you look aro...
2   At this point you only have two options, are y...
3   In general, cell phones are a huge distraction...
4   In comparison, many other people may disagree ...
5   In my opinion, there shouldn't necessarily be ...
6   In conclusion, if causing harm and chaos is th...
7   Now back to the questions. Are you willing to ...
8                                          Work Cited
9   Ellis, Baldwin. "Pros of Driving & Cell Phone ...
10                                         Work Cited
11  Hilliard, Justin, et al. "Why You Really Shoul...
12                                         Work Cited
13  "Pros And Cons Of Using A Hands Free Mobile Ph...


In [23]:
def cleanText2(item):
  item2 = ""
  for ch in item:
      if ch == ' ' or (ch >= 'a' and ch <= 'z') or (ch >= 'A' and ch <= 'Z') or (ch >= '0' and ch <= '9'):
          if ch >= 'A' and ch <= 'Z':
              item2 += ch.lower()
          else:
              item2 += ch
  return item2

In [24]:
def retrieveEssay(essay_id):
  essay_text = retrieveWordage(essay_id)
  if essay_text is None:
        return None
  essay_count = len(essay_text)
  essay = ""
  for i in range(0,essay_count-1):
      essay += cleanText2(essay_text.iloc[i,0])
  return essay

In [25]:
def retrieveRawEssay(essay_id):
  essay_text = retrieveWordage(essay_id)
  if essay_text is None:
      return None
  essay_count = len(essay_text)
  essay = ""
  for i in range(0,essay_count-1):
      item = essay_text.iloc[i,0]
      essay += item
  return essay

In [26]:
essay_test = retrieveEssay("423A1CA112E2")
print(essay_test)

phonesmodern humans today are always on their phone they are always on their phone more than 5 hours a day no stop all they do is text back and forward and just have group chats on social media they even do it while driving they are some really bad consequences when stuff happens when it comes to a phone some certain areas in the united states ban phones from class rooms just because of itwhen people have phones they know about certain apps that they have apps like facebook twitter instagram and snapchat so like if a friend moves away and you want to be in contact you can still be in contact by posting videos or text messages people always have different ways how to communicate with a phone phones have changed due to our generationdriving is one of the way how to get around people always be on their phones while doing it which can cause serious problems thats why theres a thing thats called no texting while driving thats a really important thing to remember some people still do it beca

In [27]:
from tensorflow.data import Dataset

In [28]:
from tensorflow.data import TextLineDataset

In [29]:
def retrieveDataset(essay_id):
  essay_array = []
  essay_text = retrieveWordage(essay_id)
  if essay_text is None:
    return None
  essay_count = len(essay_text)
  #essay = ""
  for i in range(0,essay_count-1):
      essay_array.append(cleanText2(essay_text.iloc[i,0]))
  #essay = essay_text.iloc[0,0]
  dataset = Dataset.from_tensor_slices(essay_array)
  return dataset

In [30]:
test_dataset = retrieveDataset("423A1CA112E2")
print(test_dataset)
for element in test_dataset:
    print(element)

<TensorSliceDataset shapes: (), types: tf.string>
tf.Tensor(b'phones', shape=(), dtype=string)
tf.Tensor(b'modern humans today are always on their phone they are always on their phone more than 5 hours a day no stop all they do is text back and forward and just have group chats on social media they even do it while driving they are some really bad consequences when stuff happens when it comes to a phone some certain areas in the united states ban phones from class rooms just because of it', shape=(), dtype=string)
tf.Tensor(b'when people have phones they know about certain apps that they have apps like facebook twitter instagram and snapchat so like if a friend moves away and you want to be in contact you can still be in contact by posting videos or text messages people always have different ways how to communicate with a phone phones have changed due to our generation', shape=(), dtype=string)
tf.Tensor(b'driving is one of the way how to get around people always be on their phones whi

2022-02-25 17:38:22.544471: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [31]:
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization

In [32]:
from tensorflow.keras.layers import TextVectorization

In [33]:
# Create the layer.
vectorize_layer = TextVectorization(
 max_tokens=5000,
 output_mode='int',
 output_sequence_length=500)

In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow import string as tf_string

In [35]:
def retrieveDiscourse(essay_id,discourse_id,discourse_start,discourse_end):
  essay_text = retrieveRawEssay(essay_id)
  essay_count = len(essay_text)
  i = 0
  discourse_text = ""
  for ch in essay_text:
     #print(ch)
     if i >= discourse_start -1 and i <= discourse_end:
        discourse_text += ch
     i += 1
  return cleanText2(discourse_text)

In [36]:
test_discourse = retrieveDiscourse("423A1CA112E2",0,8,229)
print(test_discourse)
print(len(test_discourse))

odern humans today are always on their phone they are always on their phone more than 5 hours a day no stop all they do is text back and forward and just have group chats on social media they even do it while driving th
219


In [37]:
# https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/nlp/ipynb/pretrained_word_embeddings.ipynb#scrollTo=Q6j-LRMFc-AR

In [38]:
#!wget http://nlp.stanford.edu/data/glove.6B.zip
#!unzip -q glove.6B.zip

In [39]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.initializers import Constant
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras import Model

In [40]:
essays_group = train_df.groupby(['id'])

In [41]:
class_dict = {}
samples = []
labels = []
class_index = 0
for essay_id, essay_pd in essays_group:  
  essay = retrieveEssay(essay_id)
  if essay is None:
    break
  discourse_count = len(essay_pd)
  for i in range(0,discourse_count):
    item = essay_pd.iloc[i,]
    discourse_id = item['discourse_id']
    discourse_start = item['discourse_start']
    discourse_end = item['discourse_end']
    discourse_type = item['discourse_type']
    discourse_type = discourse_type.lower()
    if class_dict.get(discourse_type) == None:
      class_name_index = class_index
      class_dict[discourse_type] = class_name_index
      class_index += 1
    else:
      class_name_index = class_dict.get(discourse_type)
    discourse = retrieveDiscourse(essay_id,discourse_id,discourse_start,discourse_end)
    samples.append(discourse)
    labels.append(class_name_index)

In [42]:
essay_array = []
samples_len = len(samples)
for i in range(0,samples_len):
  essay_text = samples[i]
  if essay_text != "":
      essay_array.append(essay_text)
vocab = Dataset.from_tensor_slices(essay_array)

In [43]:
# Now that the vocab layer has been created, call `adapt` on the text-only
# dataset to create the vocabulary. You don't have to batch, but for large
# datasets this means we're not keeping spare copies of the dataset.
vectorize_layer.adapt(vocab)

2022-02-25 17:46:32.511654: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [44]:
voc = vectorize_layer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [45]:
print(word_index)

{'': 0, '[UNK]': 1, 'the': 2, 'to': 3, 'a': 4, 'and': 5, 'of': 6, 'that': 7, 'in': 8, 'is': 9, 'you': 10, 'it': 11, 'they': 12, 'be': 13, 'for': 14, 'have': 15, 'are': 16, 'students': 17, 'not': 18, 'on': 19, 'can': 20, 'or': 21, 'i': 22, 'their': 23, 'would': 24, 'people': 25, 'if': 26, 'this': 27, 'with': 28, 'do': 29, 'because': 30, 'will': 31, 'more': 32, 'school': 33, 'but': 34, 'as': 35, 'we': 36, 'your': 37, 'at': 38, 'them': 39, 'get': 40, 'could': 41, 'there': 42, 'from': 43, 'what': 44, 'when': 45, 'so': 46, 'one': 47, 'all': 48, 'should': 49, 'by': 50, 'help': 51, 'an': 52, 'like': 53, 'was': 54, 'time': 55, 'also': 56, 'some': 57, 'about': 58, 'just': 59, 'make': 60, 'car': 61, 'cars': 62, 'many': 63, 'other': 64, 'who': 65, 'think': 66, 'has': 67, 'how': 68, 'work': 69, 'student': 70, 'electoral': 71, 'up': 72, 'dont': 73, 'out': 74, 'even': 75, 'college': 76, 'its': 77, 'he': 78, 'vote': 79, 'good': 80, 'know': 81, 'want': 82, 'may': 83, 'our': 84, 'my': 85, 'than': 86, '

In [48]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

In [51]:
path_to_glove_file = os.path.join(glove_directory,"glove.6B.100d.txt")

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [52]:
# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 4801 words (199 misses)


In [53]:
embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=Constant(embedding_matrix),
    trainable=False,
)

In [54]:
class_names = []
for key in class_dict:
  print(key)
  class_names.append(key)

position
evidence
claim
counterclaim
rebuttal
concluding statement
lead


In [55]:
print(class_dict)
print(class_names)

{'position': 0, 'evidence': 1, 'claim': 2, 'counterclaim': 3, 'rebuttal': 4, 'concluding statement': 5, 'lead': 6}
['position', 'evidence', 'claim', 'counterclaim', 'rebuttal', 'concluding statement', 'lead']


In [56]:
# Extract a training & validation split
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
test_samples = samples[-num_validation_samples:]

In [57]:
print(len(train_samples))
print(len(test_samples))

115435
28858


In [59]:
train_labels = labels[0:len(train_samples)]
test_labels = labels[len(train_samples):(len(train_samples)+len(test_samples))]

In [ ]:
# The class names should be the categories we are looking for.

In [62]:
int_sequences_input = Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(len(class_names), activation="softmax")(x)
model = Model(int_sequences_input, preds)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 100)         500200    
_________________________________________________________________
conv1d_6 (Conv1D)            (None, None, 128)         64128     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, None, 128)         82048     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, None, 128)         8204

In [64]:
x_train = vectorize_layer(np.array([[s] for s in train_samples])).numpy()
x_test = vectorize_layer(np.array([[s] for s in test_samples])).numpy()

In [65]:
y_train = np.array([[s] for s in train_labels])
y_test = np.array([[s] for s in test_labels])

In [66]:
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
)

In [67]:
#model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_test, y_test))
model.fit(x_train, y_train, batch_size=128, epochs=20)
#model.fit(x_train,y_train)

Epoch 1/20
902/902 [==============================] - 224s 247ms/step - loss: 1.1156 - acc: 0.6455
Epoch 2/20
902/902 [==============================] - 223s 248ms/step - loss: 0.9668 - acc: 0.6879
Epoch 3/20
902/902 [==============================] - 228s 253ms/step - loss: 0.9017 - acc: 0.7083
Epoch 4/20
902/902 [==============================] - 225s 250ms/step - loss: 0.8620 - acc: 0.7187
Epoch 5/20
902/902 [==============================] - 226s 250ms/step - loss: 0.8349 - acc: 0.7272
Epoch 6/20
902/902 [==============================] - 222s 246ms/step - loss: 0.8143 - acc: 0.7328
Epoch 7/20
902/902 [==============================] - 222s 247ms/step - loss: 0.7956 - acc: 0.7381
Epoch 8/20
902/902 [==============================] - 225s 249ms/step - loss: 0.7793 - acc: 0.7435
Epoch 9/20
902/902 [==============================] - 226s 251ms/step - loss: 0.7677 - acc: 0.7459
Epoch 10/20
902/902 [==============================] - 226s 251ms/step - loss: 0.7590 - acc: 0.7497
Epoch 11/

In [68]:
y_result = model.predict(x_test)

In [ ]:
print(y_result)

In [69]:
yt0 = y_test[1]

In [72]:
y_rounded = np.round(y_result,1)

In [73]:
y0 = y_rounded[1]

In [74]:
print(y0)

[0. 1. 0. 0. 0. 0. 0.]


In [75]:
print(yt0)

[1]


In [76]:
print(len(y_rounded))
print(len(y_test))

28858
28858


In [77]:
for i in range(0,10):
    y_out = y_rounded[i]
    yt = y_test[i]
    print(y_out)
    print(yt)

[0.2 0.1 0.6 0.  0.  0.  0. ]
[2]
[0. 1. 0. 0. 0. 0. 0.]
[1]
[0. 0. 1. 0. 0. 0. 0.]
[1]
[0.  0.  0.5 0.5 0.1 0.  0. ]
[2]
[0.  0.6 0.2 0.  0.1 0.  0. ]
[1]
[0.1 0.2 0.1 0.  0.  0.6 0. ]
[2]
[0.1 0.2 0.1 0.  0.  0.6 0. ]
[1]
[0.9 0.  0.1 0.  0.  0.  0. ]
[0]
[0.1 0.  0.8 0.  0.  0.  0. ]
[2]
[0.  0.3 0.6 0.  0.1 0.  0. ]
[2]


In [ ]:
# Create Statistics about Where Discourses are in Essays
# To do this we first create a pandas dataframe 'essay_statistics', per essay it lists
# essay_id,essay_length (in characters),discourse_count
class_dict = {}
samples = []
labels = []
class_index = 0
length_of_essay = 0
stats = []
for essay_id, essay_pd in essays_group: 
  stats_entry = []
  stats_entry.append(essay_id)
  essay = retrieveEssay(essay_id)
  if essay is None:
    break
  length_of_essay = len(essay)
  stats_entry.append(length_of_essay)
  discourse_count = len(essay_pd)
  stats_entry.append(discourse_count)
  stats.append(stats_entry)
essay_statistics = pd.DataFrame(stats)

In [ ]:
essay_statistics.head()